In [5]:
import tensorflow as tf
from tensorflow.keras import layers as L


class CNNResUnit(tf.keras.models.Model):
  #number of cnn layers before concat. 
  #the filter size, will be a single number.
  def __init__(self, layers, num_filters, filter_size, **kwargs): 
    super().__init__(**kwargs)
    self.hidden = [L.Conv2D(num_filters, (filter_size, filter_size), activation="relu", padding="same") for _ in range(layers)]
    
  def call(self, inputs):
    x = inputs
    for layer in self.hidden:
      x = layer(x) 
    return inputs + x 


class DenseResUnit(tf.keras.models.Model):
  def __init__(self, layers, units, **kwargs):
    super().__init__(**kwargs)
    self.hidden = [L.Dense(units, activation="relu") for _ in range(layers)]

  def call(self, inputs):
    x = inputs 
    for layer in self.hidden:
      x = layer(x)
    return x + inputs


class ResnetModel(tf.keras.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.dense1 = L.Dense(64, activation="relu")
    self.cnnres = CNNResUnit(2, 64, 3)
    self.denseres = DenseResUnit(2, 64)
    self.out = L.Dense(10, activation="softmax")

  def call(self, inputs):
    #x = self.dense1(inputs) 
    x = self.cnnres(inputs)
    for i in range(3):
      x = self.denseres(x)
    
    x = L.Flatten()(x)
    return self.out(x)



In [6]:
import numpy as np

(x_t, y_t), (x_v, y_v) = tf.keras.datasets.mnist.load_data()
x_t , x_v  = np.expand_dims((x_t /255.),axis=-1) , np.expand_dims((x_v/255.), axis=-1) 


model = ResnetModel()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])

In [7]:
model.fit(x_t, y_t, validation_data=(x_v, y_v), epochs=20, batch_size=64)

Epoch 1/20
938/938 [==============================] - 19s 19ms/step - loss: 0.8342 - acc: 0.8411 - val_loss: 0.6273 - val_acc: 0.9031
Epoch 2/20
938/938 [==============================] - 18s 19ms/step - loss: 0.5428 - acc: 0.9180 - val_loss: 0.4918 - val_acc: 0.9408
Epoch 3/20
938/938 [==============================] - 18s 19ms/step - loss: 0.3774 - acc: 0.9533 - val_loss: 0.3926 - val_acc: 0.9568
Epoch 4/20
938/938 [==============================] - 18s 19ms/step - loss: 0.2760 - acc: 0.9679 - val_loss: 0.4142 - val_acc: 0.9661
Epoch 5/20
938/938 [==============================] - 18s 19ms/step - loss: 0.2672 - acc: 0.9734 - val_loss: 0.4373 - val_acc: 0.9691
Epoch 6/20
938/938 [==============================] - 18s 19ms/step - loss: 0.2265 - acc: 0.9782 - val_loss: 0.5371 - val_acc: 0.9691
Epoch 7/20
938/938 [==============================] - 18s 19ms/step - loss: 0.1997 - acc: 0.9826 - val_loss: 0.4952 - val_acc: 0.9738
Epoch 8/20
938/938 [==============================] - 18s 19ms

KeyboardInterrupt: ignored